# Q&A Benchmark Orchestrator - optional

The `QuestionAnsweringBenchmarkOrchestrator` can process Q&A datasets and evaluate how good a target is at answering the questions.

In [1]:
# Import necessary packages
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.datasets import fetch_wmdp_dataset
from pyrit.memory import CentralMemory
from pyrit.models import (
    QuestionAnsweringDataset,
    QuestionAnsweringEntry,
    QuestionChoice,
)
from pyrit.orchestrator import QuestionAnsweringBenchmarkOrchestrator
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.score.self_ask_question_answer_scorer import SelfAskQuestionAnswerScorer

# Initialize PyRIT (load environment files and set central memory instance)
initialize_pyrit(memory_db_type=IN_MEMORY)

In [2]:
# Set up the Azure OpenAI prompt target

objective_scorer = SelfAskQuestionAnswerScorer(chat_target=OpenAIChatTarget())

# Create the orchestrator with scorer and demo dataset
benchmark_orchestrator = QuestionAnsweringBenchmarkOrchestrator(
    objective_target=OpenAIChatTarget(),
    objective_scorer=objective_scorer,
)

question = QuestionAnsweringEntry(
            question="What is the capital of France?",
            answer_type="str",
            correct_answer="0",
            choices=[
                QuestionChoice(index=0, text="Paris"),
                QuestionChoice(index=1, text="London"),
                QuestionChoice(index=2, text="Berlin"),
                QuestionChoice(index=3, text="Madrid"),
            ],
        )




# Sends prompts associated with dataset
result = await benchmark_orchestrator.run_attack_async(question_answering_entry=question)
await result.print_conversation_async()


The orchestrator has completed the conversation and achieved the objective: 
The objective is to answer the question: What is the capital of France? with the correct answer: "0: Paris".
Any other response that does not clearly indicate either the correct index or correct answer is incorrect.

user:
Converted value: What is the capital of France?

Option 0: Paris
Option 1: London
Option 2: Berlin
Option 3: Madrid
assistant: The capital of France is:

**Option 0: Paris**
objective score: SelfAskQuestionAnswerScorer: question_answering: True : The response correctly identifies the capital of France as 'Paris' and clearly indicates the correct index, which is Option 0. Both parts of the answer fulfill the criteria laid out in the instructions, ensuring it is precise and accurate.


In [4]:
# Fetch WMDP dataset for Q/A Model Testing; this is big and can take a minute

wmdp_ds = fetch_wmdp_dataset(category="cyber")

In [5]:

# Evaluate the Q/A Model response
results = await benchmark_orchestrator.run_attacks_async(question_answering_entries=wmdp_ds.questions[:3])
for result in results:
    await result.print_conversation_async()

TypeError: QuestionAnsweringBenchmarkOrchestrator.run_attack_async() got an unexpected keyword argument 'question_ansering_entry'

You can also do analysis and benchmarking using these. To see the percentage of correct answers, you can use the `OrchestratorResult` object.

In [ ]:
success = sum(r.status == "success" for r in results)
print(f"Success rate: {success / len(results)}")
